In [15]:
import os, sys
sys.path.insert(0, os.path.abspath(".."))

import numpy as np
from sklearn import datasets

from core.notebook.variable_snapshot import VariableSnapshotSet
from core.notebook.record_event import RecordEvent

from sklearn.neighbors import KNeighborsClassifier

from core.globals import reset_notebook_state
reset_notebook_state()

In [16]:
@RecordEvent
def cell_2():
    iris_X, iris_y = datasets.load_iris(return_X_y=True)
    
    return VariableSnapshotSet({"iris_X": iris_X, "iris_y": iris_y})

In [17]:
iris_X, iris_y = cell_2()


container created for variable  iris_X
container created for variable  iris_y


In [18]:
@RecordEvent
def cell_3():
    np.random.seed(0)

In [19]:
cell_3()

In [20]:
@RecordEvent
def cell_4():
    indices = np.random.permutation(len(iris_X))
    iris_X_train = iris_X[indices[:-10]]
    iris_y_train = iris_y[indices[:-10]]
    iris_X_test = iris_X[indices[-10:]]
    iris_y_test = iris_y[indices[-10:]]
    
    return VariableSnapshotSet({"indices": indices, "iris_X_train": iris_X_train, "iris_y_train": iris_y_train, "iris_X_test": iris_X_test, "iris_y_test": iris_y_test})

In [21]:
indices, iris_X_train, iris_y_train, iris_X_test, iris_y_test = cell_4()


container created for variable  indices
container created for variable  iris_X_train
container created for variable  iris_y_train
container created for variable  iris_X_test
container created for variable  iris_y_test


In [22]:
@RecordEvent
def cell_5():
    knn = KNeighborsClassifier()
    knn.fit(iris_X_train, iris_y_train)

    knn.predict(iris_X_test)
    
    return VariableSnapshotSet({"knn": knn})

In [23]:
knn = cell_5()


container created for variable  knn


In [24]:
from core.globals import operation_events

for x in operation_events:
    print(x, "\n")
    print("input variable_snapshots: \n")
    for y in x.input_variable_snapshots:
        print(y, "\n")
    print("------------")

Operation with name cell_2 and id 0 started at 2022-09-25 23:02:32.327426 and ended at 2022-09-25 23:02:32.328637, 0.001211 s elapsed, 0 input variable snapshots 

input variable_snapshots: 

------------
Operation with name cell_3 and id 1 started at 2022-09-25 23:02:32.335195 and ended at 2022-09-25 23:02:32.335207, 1.2e-05 s elapsed, 0 input variable snapshots 

input variable_snapshots: 

------------
Operation with name cell_4 and id 2 started at 2022-09-25 23:02:32.342208 and ended at 2022-09-25 23:02:32.342472, 0.000264 s elapsed, 2 input variable snapshots 

input variable_snapshots: 

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2] 

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0

In [25]:
# Create a graph representation of the current state of the notebook
from core.notebook.create_graph import create_graph
graph = create_graph()

print("---------------------------")
print("Active nodes:")
for node in graph.active_nodes:
    print(node.vs.get_name(), node.vs.get_version(), node.vs.get_size())

print("---------------------------")
print("Edges:")
for edge in graph.edges:
    print("---------------------------")
    print("Edge:", edge.oe.cell_func_name, edge.oe.duration)
    print("sources")
    for node in edge.src.nodes:
        print("   ", node.vs.get_name(), node.vs.get_version(), node.vs.get_size())
    print("destinations")
    for node in edge.dst.nodes:
        print("   ", node.vs.get_name(), node.vs.get_version(), node.vs.get_size())

---------------------------
Active nodes:
iris_X 1 4928
iris_y 1 1312
indices 1 1312
iris_X_train 1 4608
iris_y_train 1 1232
iris_X_test 1 448
iris_y_test 1 192
knn 1 48
---------------------------
Edges:
---------------------------
Edge: cell_2 0.001211
sources
destinations
    iris_X 1 4928
    iris_y 1 1312
---------------------------
Edge: cell_3 1.2e-05
sources
destinations
---------------------------
Edge: cell_4 0.000264
sources
    iris_y 1 1312
    iris_X 1 4928
destinations
    indices 1 1312
    iris_X_train 1 4608
    iris_y_train 1 1232
    iris_X_test 1 448
    iris_y_test 1 192
---------------------------
Edge: cell_5 0.001593
sources
    iris_y_train 1 1232
    iris_X_train 1 4608
    iris_X_test 1 448
destinations
    knn 1 48


In [26]:
# Use the optimizer to find nodes to migrate
from algorithm.optimizer_greedy import OptimizerGreedy
from algorithm.optimizer_exact import OptimizerExact
from algorithm.baseline import MigrateAllBaseline
from algorithm.baseline import RecomputeAllBaseline
from algorithm.baseline import RandomBaseline

opt = OptimizerGreedy(migration_speed_bps=100000)
graph.trim_graph(opt)

print("---------------------------")
print("Nodes to recompute:")
for node in graph.nodes_to_recompute:
    print(node.vs.get_name(), node.vs.get_version(), node.vs.get_size())
    print(node.vs)

current score:  0.1408
best decrease:  0.014776000000000011
best set:  5
current score:  0.126024
best decrease:  0.06287999999999999
best set:  6
current score:  0.063144
best decrease:  0.0
best set:  4
---------------------------
Nodes to migrate:
iris_y 1 1312
iris_X 1 4928
knn 1 48
---------------------------
Nodes to recompute:
iris_X_test 1 48
iris_y_test 1 48
indices 1 48
iris_X_train 1 48
iris_y_train 1 48


In [27]:
# Migrate the dependency graph.
from core.io.filesystem_adapter import FilesystemAdapter
from core.io.migrate import migrate

migrate(graph, FilesystemAdapter())